### Tratamento das imagens em diferentes camadas

In [ ]:
from tensorflow.keras import models, layers, activations, initializers

model = models.Sequential([
    layers.Input(shape=(32,32,3)),

    layers.Resizing(32,32),
    layers.Rescaling(1./255),
    layers.RandomRotation((-0.2, 0.2)),

    layers.Conv2D(32, kernel_size=(2,2), strides=(1,1), padding='same'),           
    layers.MaxPooling2D((1,1)),
    
    layers.Conv2D(64, kernel_size=(2,2), strides=(1,1), padding='same'),
    layers.MaxPooling2D((1,1)),
    
    layers.Conv2D(128, kernel_size=(2,2), strides=(1,1), padding='same'),
    layers.MaxPooling2D((1,1)),

    layers.Flatten(),

    layers.Dense(64, activation=activations.relu, kernel_initializer=initializers.RandomNormal()),
    layers.Dropout(0.2),

    layers.Dense(64, activation=activations.relu, kernel_initializer=initializers.RandomNormal()),

    layers.Dense(24, activation=activations.softmax, kernel_initializer=initializers.RandomNormal())
])

### Configurando o optimizer

In [ ]:
from tensorflow.keras import optimizers, losses, metrics

lr = 0.001

model.compile(
    optimizer = optimizers.Adam(
        learning_rate = lr    
    ),
    loss = losses.SparseCategoricalCrossentropy(),
    metrics = [ metrics.sparse_categorical_accuracy ]
)

### Selecionando as imagens para o treinamento e teste

In [ ]:
from tensorflow.keras import utils

path = './Cifra10'
batch_size = 50

train = utils.image_dataset_from_directory(
    directory=path + '/train',
    shuffle = True,
    seed = 1,
    subset = 'training',
    validation_split = 0.1,
    image_size = (32,32),
    batch_size = batch_size
)

test = utils.image_dataset_from_directory(
    directory=path + '/test',
    shuffle = True,
    seed = 1,
    subset = 'validation',
    validation_split = 0.1,
    image_size = (32,32),
    batch_size = batch_size
)

### Realizando o treinamento do modelo

In [ ]:
from tensorflow.keras import callbacks

model_path = "./cifra10.keras"

patience = 5
epochs = 100

model.fit(
    train,
    validation_data = test,
    epochs = epochs,
    verbose = True,
    
    callbacks = [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        ),
        callbacks .ModelCheckpoint(
            filepath = model_path,
            save_weights_only = False,
            monitor = 'loss',
            mode = 'min',
            save_best_only = True
        )
    ]
)

labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']